## Content



## IMPORT LIBRARES AND PACKAGES

In [1]:
import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os, cv2


## [1] DATA PREPARATION

### [1.1] DATA LOADING

Function to load image

Setup data directory

In [2]:
IMAGE_DIRECTORY = "./data-1/cohn-kanade-images/"
DATA_LABEL_DIRECTORY = "./data-1/data_labels.csv"

#### Load Label Dataset

In [3]:
df = pd.read_csv(DATA_LABEL_DIRECTORY)

#### Load Image Dataset

In [4]:
# Initialize an empty list to store processed images
img_data_list = []

# Get a list of image file paths from the 'filepath' column in the DataFrame
img_file_list = df['filepath'].tolist()

# Iterate through each image file path
for img in img_file_list:
    input_img = cv2.imread(img)
    # Resize the image to a fixed size of 48x48 pixels
    input_img_resize=cv2.resize(input_img,(48,48))
    img_data_list.append(input_img_resize)
# Convert the list of resized images to a NumPy array
img_data = np.array(img_data_list)
# Convert the data type of pixel values to 'float32'
img_data = img_data.astype('float32')
# Normalize pixel values by scaling them to the range [0, 1]
img_data = img_data/255

# Check Shape
# 560 images.
# 48 pixels in height.
# 48 pixels in width.
# Each pixel has 3 values (RGB).
img_data.shape

KeyError: 'filepath'

### [1.2] DATA CLEANING AND PREPARE

Convert target value to number

In [ ]:
# Define mapping conversion
target_mapping = {'negative': 0, 'positive': 1, 'surprise': 2}
df['high_level_emotion_num'] = df['high_level_emotion'].map(target_mapping)

## [2] FEATURE EXTRACTION

In [ ]:
from keras.utils import to_categorical
from sklearn.utils import shuffle

In [ ]:
names = ['negative','positive','surprise']

def getLabel(id):
    return ['negative','positive','surprise'][id]

Extract indices 

In [ ]:
num_classes = 3
labels = df['high_level_emotion_num'].to_list()
Y = to_categorical(labels, num_classes=num_classes)
x,y = shuffle(img_data,Y, random_state=2)
x.shape, y.shape

((560, 48, 48, 3), (560, 3))